# Red convolucional - Inception

1. Introducción arquitectura VGG_16.
2. Provision de imágenes a la arquitectura. Pre procesado de imágenes.
    * Estructura listado
3. Implementación arquitectura.
    * Keras
    * Capas
    * Arquitectura pre entrenada - Imagenet 
4. Transferencia de aprendizaje: descriptores profundos.
    * Reducción dimensionalidad
    * Clasificación
    * Medidas de desempeño
4. Transferencia de aprendizaje: Fine tuning.
    * Medidas de desempeño
5. Mejoras
    * Provisión de imágenes: hd5py - TFrecords
    * Implementación arquitectura: Tensorflow
6. Resultados - Tablas. Resumen
7. Discusión resultados

## Introducción arquitectura VGG_16.

In [ ]:
# http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-16.svg

## Provision de imágenes a la arquitectura. Pre procesado de imágenes.

In [3]:
# conda install -c conda-forge keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import numpy as np

Explicación

In [4]:
#https://blog.keras.io/img/imgclf/vgg16_original.png
#fuente https://blog.keras.io/
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())

#top layer of the VGG net
model.add(Dense(4096, activation=None))
model.add(Dropout(0.5))
model.add(Dense(4096, activation=None))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

Explicacion

In [ ]:
# https://github.com/fchollet/deep-learning-models/releases
# format h5py
model.load_weights("pretrained-model/Keras-VGG16/vgg16_weights_tf_dim_ordering_tf_kernels.h5")

Explicacion

In [ ]:
optimizer = SGD()
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

Explicacion

In [ ]:
# check this cell!!!!
from keras.applications.vgg16 import VGG16
# prebuild model with pre-trained weights on imagenet
model = VGG16(weights='imagenet', include_top=True)
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

In [ ]:
Extracción descriptores profundos

In [ ]:
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [13]:
# pre-built and pre-trained deep learning VGG16 model
base_model = VGG16(weights='imagenet', include_top=True)
for i, layer in enumerate(base_model.layers):
    print (i, layer.name, layer.output_shape)

0 input_8 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


Ejemplo con una fotografía deep features

In [ ]:
# we need pre processing function in another ipython
# %run '../dataset/dataset-v0.1-spa.ipynb' # improve this point

In [4]:
# this code is on dataset-v0.1-spa.ipynb

# 1.- get the imgList structure
import os
import re

PATH_TO_IMG = "../img-raw"

imgList = []

for imgCategory in imgCategories:
    imgFolder = os.path.join(PATH_TO_IMG, imgCategory)
    
    for img in os.listdir(imgFolder):
        if img.endswith(".tif"): # just in case there are other kind of files like .db
            
            # windows users: backslash management method replace is used [it is not mandatory tbh]
            #imgList.append(os.path.join(imgFolder, img).replace("\\","/"))
            
            # linux users:
            imgList.append(os.path.join(imgFolder, img))


# 2.- pre process the image
from PIL import Image # imported before

# http://effbot.org/imagingbook/introduction.htm

def img_preprocessing(arch, img, width = 150, height = 150):
    
    imgAux = Image.open(img)
    if(arch=="VGG16"):
        return imgAux.resize((224,224))

    #add new archs
    
    return img

imgExample = img_preprocessing("VGG16", imgList[0])

fc1 vs fc2: la 1 es una capa en la que sus pesos no estan tan ajustados al cojunto de dato como la segunda

In [ ]:
# fc1
model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)

dfImgExample_fc1  = image.img_to_array(imgExample)
dfImgExample_fc1  = np.expand_dims(dfImgExample_fc1 , axis=0)
dfImgExample_fc1  = preprocess_input(dfImgExample_fc1 )

# get the features from this block
features_fc1  = model.predict(dfImgExample_fc1) # numpy.ndarray

# fc2
model = Model(input=base_model.input, output=base_model.get_layer('fc2').output)

dfImgExample_fc2 = image.img_to_array(imgExample)
dfImgExample_fc2 = np.expand_dims(dfImgExample_fc2, axis=0)
dfImgExample_fc2 = preprocess_input(dfImgExample_fc2)

# get the features from this block
features_fc2 = model.predict(dfImgExample_fc2)


print("Layer fc1: ",features_fc1.shape)
print("Layer fc2: ",features_fc2.shape)

feature map 2x4096

In [ ]:
# example with 2 images
exampleList = []
exampleList.append(imgList[0])
exampleList.append(imgList[1])

# run the whole imgList - extraction of fc1 layer
from PIL import Image # imported before

# structure 
ndeepfeatures = 4096 # get the shape from the model instead
dfmap = np.empty((len(exampleList),ndeepfeatures)) # just 2 images
imgLabels = [] # check if it is better in another place

# imgList deep feature map
for i, img in enumerate(exampleList):
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    
    # pre procesado
    aux = img_preprocessing("VGG16", img)
    
    aux = image.img_to_array(aux)
    aux = np.expand_dims(aux, axis=0)
    aux = preprocess_input(aux)

    # get the features from this block
    aux  = model.predict(aux)
    
    # fill the structure
    dfmap[i,:] = np.squeeze(aux, axis=0)
    imgLabels.append(re.split('_\d+',img.split('/')[2])[0])

Se guardarán los resultados en unos ficheros para no tener que volver a repetir este proceso que necesita mucho tiempo

In [ ]:
# check the dimensions of the structures created
print(type(dfmap))
print(dfmap.shape)
print(type(imgLabels))

In [ ]:
import pickle
pickle.dump(dfmap, open('feature-maps/VGG16-example_dfmap', 'wb'))
pickle.dump(imgLabels, open('feature-maps/VGG16-example_imgLabels', 'wb'))

In [ ]:
Procesamiento de todo el conjunto de imágenes - deep features

In [ ]:
# run the whole imgList - extraction of fc1 layer
from PIL import Image # imported before

# structure 
ndeepfeatures = 4096 # get the shape from the model instead
dfmap = np.empty((len(imgList),ndeepfeatures))
imgLabels = [] # check if it is better in another place

# imgList deep feature map
for i, img in enumerate(imgList):
    if (i == 1000) or (i == 2000) or (i == 3000) or (i == 4000):
        print("Processing image nº %s:" % i)
        
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    
    # pre procesado
    aux = img_preprocessing("VGG16", img)
    
    aux = image.img_to_array(aux)
    aux = np.expand_dims(aux, axis=0)
    aux = preprocess_input(aux)

    # get the features from this block
    aux  = model.predict(aux)
    
    # fill the structure
    dfmap[i,:] = np.squeeze(aux, axis=0)
    imgLabels.append(re.split('_\d+',img.split('/')[2])[0])


In [ ]:
Guardamos los resultados 

In [ ]:
# hd5py (revisar)
pickle.dump(dfmap, open('feature-maps/VGG16-dfmap', 'wb'))
pickle.dump(imgLabels, open('feature-maps/VGG16-imgLabels', 'wb'))

In [ ]:
# SVM - sin reduccion de features

In [ ]:
#Using the bottleneck features of a pre-trained network
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
Entrenamiento de las últimas capas Fine tuning

In [ ]:
# transformacion de imgList a dataframe ¿?

In [ ]:
# https://blog.keras.io/img/imgclf/vgg16_modified.png

In [2]:
# check the imports
from keras.applications.vgg16 import VGG16
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import numpy as np

Using TensorFlow backend.


In [ ]:
# paths


In [24]:
# bottle neck --> es como la 1 parte
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# create the base pre-trained model. Exclude the top !!
base_model_bn = VGG16(weights='imagenet', include_top=False)
# all layers will be non-trainable (weights will not be updated)
for layer in base_model_bn.layers:
    layer.trainable = False

for i, layer in enumerate(base_model_bn.layers):
    print("Layer %d: %s, trainable - %s" % (i, layer.name, layer.trainable))
    print("Layer output: ",layer.output_shape)

print(base_model_bn.summary())

Layer 0: input_15, trainable - False
Layer output:  (None, None, None, 3)
Layer 1: block1_conv1, trainable - False
Layer output:  (None, None, None, 64)
Layer 2: block1_conv2, trainable - False
Layer output:  (None, None, None, 64)
Layer 3: block1_pool, trainable - False
Layer output:  (None, None, None, 64)
Layer 4: block2_conv1, trainable - False
Layer output:  (None, None, None, 128)
Layer 5: block2_conv2, trainable - False
Layer output:  (None, None, None, 128)
Layer 6: block2_pool, trainable - False
Layer output:  (None, None, None, 128)
Layer 7: block3_conv1, trainable - False
Layer output:  (None, None, None, 256)
Layer 8: block3_conv2, trainable - False
Layer output:  (None, None, None, 256)
Layer 9: block3_conv3, trainable - False
Layer output:  (None, None, None, 256)
Layer 10: block3_pool, trainable - False
Layer output:  (None, None, None, 256)
Layer 11: block4_conv1, trainable - False
Layer output:  (None, None, None, 512)
Layer 12: block4_conv2, trainable - False
Layer ou

In [19]:
# train and validation sets from imgList y labels
import numpy as np
import re

imgList = []

for imgCategory in imgCategories:
    imgFolder = os.path.join(PATH_TO_IMG, imgCategory)
    
    for img in os.listdir(imgFolder):
        if img.endswith(".tif"): # just in case there are other kind of files like .db
            
            # windows users: backslash management method replace is used [it is not mandatory tbh]
            #imgList.append(os.path.join(imgFolder, img).replace("\\","/"))
            
            # linux users:
            imgList.append(os.path.join(imgFolder, img))




imgLabels = []

for img in imgList:
    imgLabels.append(re.split('_\d+',img.split('/')[2])[0])

dataSet = []
for i, movida in enumerate(imgLabels):
    dataSet[i-1] =imgLabels[i-1] + imgList[i-1]


# train_set_bn
# validation_set_bn



IndexError: list assignment index out of range

In [ ]:
# better with pickle?

# prediction with train and validation sets

train_features_bn = base_model_bn.predict(train_set_bn)
np.save(open('train_features_bn.npy', 'w'), train_features_bn)

validation_features_bn = base_model_bn.predict(validation_set_bn)
np.save(open('validation_features_bn.npy', 'w'), validation_features_bn)

In [ ]:
# get the data saved in npys mandatory¿?
# de train_features hay que extraer train_data y train_labels
# de validation_features hay que estraer validation_data y validation_labels






In [ ]:
top_model_bn = Sequential()
top_model_bn.add(Flatten(input_shape=model.output_shape[1:]))
top_model_bn.add(Dense(4096, activation='relu'))
top_model_bn.add(Dropout(0.5))
top_model_bn.add(Dense(4096, activation='relu'))
top_model_bn.add(Dropout(0.5))
top_model_bn.add(Dense(8, activation='softmax'))

# en la 1 parte
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=sgd, loss='categorical_crossentropy')


#check the args
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

#
bn_result = model.fit(train_data, train_labels, epochs=epochs,\\
                      batch_size=batch_size, validation_data=(validation_data, validation_labels))

In [ ]:
# comparar la estructura que tiene los labels y son los reales con los predichos por el modelo en bn_result

# matriz confusion 8x8

In [ ]:
# segunda parte fine tuning

In [ ]:
# repasar con cuidad la forma de cargar y contruir el modelo sobre todo los pesos

'''
igual me tienes que explicar la diferencia entre usar los pesos de imagenet vs estos pesos

yo tengo estos pesos

vgg16_weights_tf_dim_ordering_tf_kernels.h5
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


chollet hace esto

link original https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

en github https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975 linea 69


carga los pesos de top_model sin haber unido esa capa con las convolucionales vgg16 no_top = true
la variable de esos pesos es top_model_weights_path = 'fc_model.h5' a que se corresponde este fichero
una vez cargados si que une esa capa con el resto de la convnet para luego poner la base de la convnet
como no entrenable para fijar los pesos y entrenar la capa top

por lo que tenemos base con los pesos de imagenet no entrenable + top con esos pesos desconocidos entrenable

una vez hecho esto compila el modelo con
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

'''
'''





'''

In [ ]:
# https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069

# PUEDE SER QUE SEA LA MISMA EN AMBAS APROXIMACIONES BOTTLE NECK Y FINE TUNING
# add the top layer (classifier) to the conv-net
# top_model_bn = Sequential()
# top_model_bn.add(Flatten(input_shape=model.output_shape[1:]))
# top_model_bn.add(Dense(4096, activation='relu'))
# top_model_bn.add(Dropout(0.5))
# top_model_bn.add(Dense(4096, activation='relu'))
# top_model_bn.add(Dropout(0.5))
# top_model_bn.add(Dense(8, activation='softmax'))

# add the last layer to the bottle neck model
# https://github.com/flyyufelix/cnn_finetune/blob/master/vgg16.py



# https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975
# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
# top_model_bn.load_weights(top_model_weights_path)

# # add the model on top of the convolutional base
# model_bn.add(top_model_bn)


In [ ]:
# # all layers will be non-trainable (weights will not be updated)
# for layer in base_model_bn.layers:
#     layer.trainable = False

# for i, layer in enumerate(base_model_bn.layers):
#     print("Layer %d: %s, trainable - %s" % (i, layer.name, layer.trainable))
#     print("Layer output: ",layer.output_shape)

# print(base_model_bn.summary())

In [22]:
# fine tuning
# create the base pre-trained model. Exclude the top !!
base_model_ft = VGG16(weights='imagenet', include_top=False) 

# last conv-layer will be trainable (weights will not be updated)
for layer in base_model_ft.layers[:-4]:
    layer.trainable = False

for i, layer in enumerate(base_model_ft.layers):
    print("Layer %d: %s, trainable - %s" % (i, layer.name, layer.trainable))
    print("Layer output: ",layer.output_shape)

print(base_model_ft.summary()) # the architecture is the same

Layer 0: input_13, trainable - False
Layer output:  (None, None, None, 3)
Layer 1: block1_conv1, trainable - False
Layer output:  (None, None, None, 64)
Layer 2: block1_conv2, trainable - False
Layer output:  (None, None, None, 64)
Layer 3: block1_pool, trainable - False
Layer output:  (None, None, None, 64)
Layer 4: block2_conv1, trainable - False
Layer output:  (None, None, None, 128)
Layer 5: block2_conv2, trainable - False
Layer output:  (None, None, None, 128)
Layer 6: block2_pool, trainable - False
Layer output:  (None, None, None, 128)
Layer 7: block3_conv1, trainable - False
Layer output:  (None, None, None, 256)
Layer 8: block3_conv2, trainable - False
Layer output:  (None, None, None, 256)
Layer 9: block3_conv3, trainable - False
Layer output:  (None, None, None, 256)
Layer 10: block3_pool, trainable - False
Layer output:  (None, None, None, 256)
Layer 11: block4_conv1, trainable - False
Layer output:  (None, None, None, 512)
Layer 12: block4_conv2, trainable - False
Layer ou

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), \
              metrics=['accuracy'])